In [1]:
#######################
##  Library Imports  ##
#######################
import numpy as np
import h5py
import sys
import os
from numba import njit
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif']  = 'Computer Modern Roman'
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import getopt
from itertools import zip_longest
import multiprocessing as mprocs
import time as TIME
from subprocess import Popen, PIPE
from matplotlib.pyplot import cm
from functions import tc, sim_data, import_data, import_spectra_data, import_post_processing_data
from plot_functions import plot_sector_phase_sync_snaps

In [2]:
in_dir = "../Data/Test/SIM_DATA_NAVIER_RK4_FULL_N[128,128]_T[0-5]_NU[0.000003]_CFL[0.90]_u0[DECAY_TURB_ALT]_TAG[Test-Test]/"
post_file_path = in_dir + "PostProcessing_HDF_Data_SECTORS[24]_KFRAC[0.75]_TAG[Test-Test].h5"

In [23]:
# -----------------------------------------
# # --------  Read In data
# -----------------------------------------
## Read in simulation parameters
sys_vars = sim_data(in_dir)

## Read in solver data
run_data = import_data(in_dir, sys_vars)

## Read in spectra data
spec_data = import_spectra_data(in_dir, sys_vars)

## Read in post processing data
post_data = import_post_processing_data(post_file_path, sys_vars, 'file')

## Number of triad types
num_triad_types = 7

Kx


## Data Functions

In [24]:
def get_w(w_hat):
    return np.fft.irfft2(w_hat)

def get_u_hat_x(w_hat, k2inv, ky):
    return np.complex(0.0, 1.0) * k2inv * ky * w_hat

def get_u_hat_y(w_hat, k2inv, kx):
    return -np.complex(0.0, 1.0) * k2inv * kx[:, np.newaxis] * w_hat

def get_u_x(w_hat, k2inv, ky):
    return np.fft.irfft2(np.complex(0.0, 1.0) * k2inv * ky * w_hat)

def get_u_y(w_hat, k2inv, kx):
    return np.fft.irfft2(-np.complex(0.0, 1.0) * k2inv * kx[:, np.newaxis] * w_hat)

def get_nonlin_r(nonlin):
    return np.fft.irfft2(nonlin)

## Test Functions

In [25]:
def compare(a, b):
    max = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            max = np.maximum(np.absolute(a[i, j] - b[i, j]), max)
    print("All Close:\t{}\nL_inf Err:\t{}\nMax Abs Err:\t{}\n".format(np.allclose(a, b), np.linalg.norm(a - b, ord = np.inf), max))

In [26]:
t = 0

## Take Solver Fourier Space Vorticity As Ground Truth

In [27]:
py_what   = run_data.w_hat[t, :, :]
py_w      = get_w(py_what)
py_uhat_x = get_u_hat_x(py_what, run_data.k2Inv, run_data.ky)
py_uhat_y = get_u_hat_y(py_what, run_data.k2Inv, run_data.kx)
py_u_x    = get_u_x(py_what, run_data.k2Inv, run_data.ky)
py_u_y    = get_u_y(py_what, run_data.k2Inv, run_data.kx)
py_nonlin = get_nonlin_r(run_data.nonlin[t, :, :])

## Compare Solver Data With Python

In [28]:
print("Real Space Vorticity:")
compare(py_w, run_data.w[t, :, :])
print()

print("Real U_x:")
compare(py_u_x, run_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(py_u_y, run_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(py_uhat_x, run_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(py_uhat_y, run_data.u_hat[t, :, :, 1])
print()

Real Space Vorticity:
All Close:	True
L_inf Err:	7.37188088351104e-14
Max Abs Err:	2.6645352591003757e-15


Real U_x:
All Close:	True
L_inf Err:	4.3662989890336235e-15
Max Abs Err:	1.6653345369377348e-16


Real U_y:
All Close:	True
L_inf Err:	5.488665077990618e-15
Max Abs Err:	1.6653345369377348e-16


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0




## Compare Post Data with Python

In [29]:
print("Real Space Vorticity:")
compare(py_w, post_data.w[t, :, :])
print()

print("Real U_x:")
compare(py_u_x, post_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(py_u_y, post_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(py_uhat_x, post_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(py_uhat_y, post_data.u_hat[t, :, :, 1])
print()

Real Space Vorticity:
All Close:	True
L_inf Err:	7.37188088351104e-14
Max Abs Err:	2.6645352591003757e-15


Real U_x:
All Close:	True
L_inf Err:	4.3662989890336235e-15
Max Abs Err:	1.6653345369377348e-16


Real U_y:
All Close:	True
L_inf Err:	6.0021432268797525e-15
Max Abs Err:	1.3877787807814457e-16


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0




## Compare Post with Solver Data

In [31]:
print("Real Space Vorticity:")
compare(run_data.w[t, :, :], post_data.w[t, :, :])
print()

print("Real U_x:")
compare(run_data.u[t, :, :, 0], post_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(run_data.u[t, :, :, 1], post_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(run_data.u_hat[t, :, :, 0], post_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(run_data.u_hat[t, :, :, 1], post_data.u_hat[t, :, :, 1])
print()

print("ky:")
print(np.allclose(run_data.ky[:], post_data.ky[:]))
print()
print("kx:")
print(np.allclose(run_data.kx[:], post_data.kx[:]))
print()

Real Space Vorticity:
All Close:	True
L_inf Err:	5.434541705540141e-14
Max Abs Err:	1.7763568394002505e-15


Real U_x:
All Close:	True
L_inf Err:	2.607289384393141e-15
Max Abs Err:	1.1102230246251565e-16


Real U_y:
All Close:	True
L_inf Err:	3.7192471324942744e-15
Max Abs Err:	1.1102230246251565e-16


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


ky:
True

kx:
True



In [42]:
print(run_data.ky)
print(run_data.kx)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64 -63 -62 -61 -60 -59 -58 -57
 -56 -55 -54 -53 -52 -51 -50 -49 -48 -47 -46 -45 -44 -43 -42 -41 -40 -39
 -38 -37 -36 -35 -34 -33 -32 -31 -30 -29 -28 -27 -26 -25 -24 -23 -22 -21
 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3
  -2  -1]


In [22]:
t = 2
# for i in range(run_data.u_hat.shape[1]):
i = 1
for j in range(run_data.u_hat.shape[2]):
    print("uh_x[{},{}] {:1.10f} {:1.10f}I\tuh_y[{},{}] {:1.10f} {:1.10f}I".format(i, j, np.real(run_data.u_hat[t, i, j, 0]), np.imag(run_data.u_hat[t, i, j, 0]), i, j, np.real(run_data.u_hat[t, i, j, 1]), np.imag(run_data.u_hat[t, i, j, 1])))

uh_x[1,0] 0.0000000000 0.0000000000I	uh_y[1,0] -48.6758124140 -6.7951102323I
uh_x[1,1] -16.1296414438 56.4304077797I	uh_y[1,1] 16.1296414438 -56.4304077797I
uh_x[1,2] -201.8646221000 18.7669329305I	uh_y[1,2] 100.9323110500 -9.3834664652I
uh_x[1,3] -74.0181953378 -5.1244201528I	uh_y[1,3] 24.6727317793 1.7081400509I
uh_x[1,4] 83.1672088434 108.6145438282I	uh_y[1,4] -20.7918022108 -27.1536359571I
uh_x[1,5] 155.5004689000 -123.9583569156I	uh_y[1,5] -31.1000937800 24.7916713831I
uh_x[1,6] 61.6816218916 32.6154071349I	uh_y[1,6] -10.2802703153 -5.4359011892I
uh_x[1,7] 103.1016448002 -99.4162898344I	uh_y[1,7] -14.7288064000 14.2023271192I
uh_x[1,8] -22.9460058734 58.1872861418I	uh_y[1,8] 2.8682507342 -7.2734107677I
uh_x[1,9] 76.1886831254 -12.3692549041I	uh_y[1,9] -8.4654092362 1.3743616560I
uh_x[1,10] 37.0032939317 3.5456219382I	uh_y[1,10] -3.7003293932 -0.3545621938I
uh_x[1,11] -59.8409536013 -10.0217719012I	uh_y[1,11] 5.4400866910 0.9110701728I
uh_x[1,12] 20.4368248449 -19.9686012479I	uh_y[